In [7]:
"""
Update Parameters Here
"""
COLLECTION_NAME = "8_BIT_UNIVERSE"
CONTRACT = "0xaAe71BBbaA359BE0d81D5CBc9B1E88A8b7c58a94"

In [8]:
"""
@author: mdigi14
"""
from typing import Any, Dict, List, Optional

import pandas as pd

from honestnft_utils import config
from honestnft_utils import constants
from honestnft_utils import opensea

"""
Helper Functions
"""


def get_all_sales(token_id: Optional[int] = None) -> List[Dict[str, Any]]:
    data = opensea.get_opensea_events(
        contract_address=CONTRACT, event_type="successful", token_id=token_id
    )
    return data

In [9]:
"""
Generate Dataset
"""

events = get_all_sales()
events_list = []
df = pd.json_normalize(events)

# Keep only the columns we want
df = df[
    [
        "asset.token_id",
        "total_price",
        "transaction.timestamp",
    ]
]
df["total_price"] = df["total_price"].apply(lambda x: float(x) / constants.ETHER_UNITS)

# Rename columns
df.columns = [
    "token_id",
    "price",
    "date_sold",
]

# drop NA values
df.dropna(subset=["token_id"], inplace=True)

# keep first sale of a token only
df = df.sort_values(by=["date_sold", "token_id"]).drop_duplicates(
    subset=["token_id"], keep="first"
)

df.to_csv(
    f"{config.FIRST_FLIP_REVENUE_FOLDER}/{COLLECTION_NAME}_first_sale_revenue.csv",
    index=False,
)